In [1]:
# Import libraries
import requests
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import os, os.path as osp
import re

In [2]:
DATA_FOLDER = 'data'
SAMPLE_DATA_FOLDER = DATA_FOLDER + '/htmlSample/'

#SAMPLE_DATA_FOLDER='htmlSample/'
SAMPLE_FILE='data/htmlSample/ff6da2b8d426c56ae77beda595bdcfea.html' #Recipe
SAMPLE_FILE_2='data/htmlSample/ff727d984c9c0048def173c4c97ab52e.html'
SAMPLE_FILE_3='data/htmlSample/ff6f004f00135caa7d6f91b27cbd3a4a.html'
SAMPLE_FILE_4='data/htmlSample/ff70a9eb4e8de4c2f72f281993ab7a55.html'
SAMPLE_FILE_5='data/htmlSample/ff7208a2993418a1b6d50c88b279f229.html'
SAMPLE_FILE_6='data/sample_400/00afe1a62e4d553763db9a9624636141.html'

DATA_FOLDER_EXPANDED = DATA_FOLDER + '/sample_400/'

In [3]:
#remove first letter to avoid capitalization issues?
meat_products = ['meat', 'beef','pork', 'chicken', 'fish', 'lamb','ham', 'turkey','steak','sirloin','veal','hamburger']
units = ['pounds','grams','oz','ounces','kg','kilograms','lbs' ]

In [4]:
# This code is just to test soup on individual html files to verify format---> it is then inserted into analyse_recipe function
with open(SAMPLE_FILE_5) as f:
    page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    ing_wrap_1=soup.find_all('li', class_="ingredient")
    ing_amnts=[]
    ings=[]
    for i in ing_wrap_1:
                ing_amnts.append((i.find('span',class_='value').text+ ' '+i.find('span',class_='type').text))
                ings.append(i.find('span', class_='name').text)
                #print(i)
                #print('-----')
    #print(ing_wrap_1)
    print(ing_amnts)
    print(ings)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbd in position 22226: invalid start byte

In [ ]:
#This code is just to test analyse_recipe function on individual html files
with open(SAMPLE_FILE_5) as f:
    page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    
    ingredient_wrapper, is_recipe, ingredients, ingredient_amounts=analyse_recipe(soup,page)
    print(ingredients)

In [5]:
#to test with meat ingredients
ingredients.append('4 1/3 lbs beef')
#ingredient_amounts.append('4 1/3 lbs')
print(ingredient_amounts)
print(ingredients)

NameError: name 'ingredients' is not defined

### Necessary functions:

In [6]:
def convert_to_kg(quant, unit):
    if (unit=='kilogram') or (unit=='kg'):
        amnt_kg=quant
        #print(quant,'kg')
    elif (unit=='pound') or (unit=='lb') or (unit=='lbs') or (unit=='pounds'):
        amnt_kg=quant/2.205
        #print(amnt_kg,'kg')
    elif(unit=='g') or (unit=='gram') or (unit =='grams'):
        amnt_kg=quant/1000
        #print(amnt_kg,'kg')     
    elif(unit=='oz') or (unit=='ounce'):
        amnt_kg=quant/35.274
        #print(amnt_kg, 'kg')
    return(amnt_kg)
    
convert_to_kg(1,'oz')

0.02834949254408346

In [7]:
value = "a string 1/2 with 5 fractions like 2½ in it"
def check_quantity(quant_str):
    #Cleans input string and extracts numerical values
    #Outputs cleaned string, array of numerical values and sum of numerical values
    quant_str=quant_str.replace("½",".5")
    quant_str=quant_str.replace("1/2",".5")
    quant_str=quant_str.replace("1/3", '.33')
    quant_str=quant_str.replace('1/4','.25')
    quant_str=quant_str.replace('3/4','.75')
    quant_vals=re.findall(r"[-+]?\d*\.\d+|\d+", quant_str)
    total_quant=np.sum([float(i) for i in quant_vals])
    return quant_str, quant_vals, total_quant
a,b,c=check_quantity(value)
print('amount ',b)
print('total = ', c)

amount  ['.5', '5', '2.5']
total =  8.0


In [8]:
def extract_amount(ings_in, ing_amnt_in,meat_products_in):
    """
    Inputs: 
    ings_in= list of ingredients (and quantities)
    ing_amnt_in =  list of quantities corresponding to ingredients (empty if quanitites are included in ings_in)
    meat_products_in = list of products that we are searching for
    
    Outputs:
    meat_ingredients_full = list of meat ingredients (full string)
    meat_ingredients_base = list of meat ingredients (from base string meat_products)
    ing_amnt_out = list of corresponding quanities of meat ingredients in kg (=0 if unit not recognized)
    contains_meat = boolean (True if 1+ ingredients are recognized from meat_products list)
    
    """

    meat_ingredients_full = []
    meat_ingredients_base = []
    ing_amnt_out=[]
    contains_meat=False
    
    #Find meat ingredients
    for i in ings_in:
        for meat_product in meat_products_in:
            if i != None:
                #are any of the meat products present in the ingredients? (ignoring capitals with casefold)
                if meat_product in i.casefold(): 
                    contains_meat=True
                    meat_ingredients_full.append(i.casefold()) 
                    meat_ingredients_base.append(meat_product)

    #extract amount from string if amount is not directly available in ing_amnt_in
    if not ing_amnt_in: #if amount is empty 
        for meat_i in meat_ingredients_full:
            meat_i_quant_kg=0
            
            #print('------------')
            #print('Ingredient: ', meat_i) 
            meat_i, quantity_vals, total_quantity=check_quantity(meat_i) #pass string, return cleaned string and total quantity

            
            #find units and convert to kg
            for u in units: 
                if u in meat_i:
            #        print('Quantity in '+ u + ' converted to ')
                    meat_i_quant_kg = convert_to_kg(total_quantity,u)
            ing_amnt_out.append(meat_i_quant_kg)
            #if meat_i_quant_kg==0:
            #    print('Units not recognized for: '+meat_i)
                
                
    else:# if amount is directly available through ing_amnt
        #print('Amount available')
        for meat_i in meat_ingredients_full:
            meat_i_quant_kg=0
            #print('------------')
            #print('Ingredient: ', meat_i)
            
            #get index of ingredient in meat_ingredients_full
            meat_amount=ing_amnt_in[ings_in.index(meat_i)]
            meat_amount, quantity_vals, total_quantity=check_quantity(meat_amount)
            
            #find units and convert to kg
            for u in units: 
                if u in meat_amount.casefold():
                    meat_i_quant_kg = convert_to_kg(total_quantity,u)
                    #print('Quantity in '+ u + ' converted to ', meat_i_quant_kg, 'kg')
            ing_amnt_out.append(meat_i_quant_kg)
            #if meat_i_quant_kg==0:
                #print('Units not recognized for: '+meat_amount+meat_i)
    return meat_ingredients_full, meat_ingredients_base, ing_amnt_out, contains_meat


    
extract_amount(ingredients,ingredient_amounts, meat_products)


NameError: name 'ingredients' is not defined

In [ ]:
def analyse_recipe(soup, page):
    """
    Input:
    soup
    page
    
    Output:
    ing_wrap =  wrapper for ingredients
    is_recipe = boolean (True if website is found in title)
    ings = list of ingredients
    ing_amnts = list of ingredient quantities if directly available through the soup (ingredients and quantities separated)
    """
    ings = []
    ing_amnts = []
    if 'Allrecipes' in soup.title.string:
        print('This recipe is from Allrecipes')
        is_recipe = True
        ing_wrap=soup.find_all(itemprop="recipeIngredient")
        if not ing_wrap:
            print('alternative format used')
            ing_wrap=soup.find_all('li', class_="plaincharacterwrap ingredient")
            for ing in ing_wrap:
                ings.append(ing.text)
            if not ing_wrap:
                print('alternative format 2 needed for Allrecipes')
            #for ing in ing_wrap:
            #    ing_amnts.append(ing.find(itemprop='amount').text)
            #    ings.append(ing.find(itemprop='name').text)
              
    elif 'Epicurious' in soup.title.string:
        print('This recipe is from Epicurious')
        is_recipe=True
        ing_wrap=soup.find('div', id="ingredients")
        for ing in ing_wrap:
            ings.append(ing.string)
    
    elif 'Food Network' in soup.title.string:
        print('This recipe is from FoodNetwork')
        is_recipe=True
        ing_wrap=soup.find_all('li',class_='ingredient')
        for ing in ing_wrap:
            ings.append(ing.text)
        
    elif 'Food.com' in soup.title.string:
        print('This recipe is from Food.com')
        is_recipe=True
        ing_wrap=soup.find_all('li', class_="ingredient")
        for ing in ing_wrap:
            ing_amnts.append((ing.find('span',class_='value').text+ ' '+ing.find('span',class_='type').text))
            ings.append(ing.find('span', class_='name').text)
    
    elif 'Betty Crocker' in soup.title.string:
        print('This recipe is from Betty Crocker')
        is_recipe = True
        ing_wrap=soup.find_all('dl', class_='ingredient')
        for ing in ing_wrap:
            ings.append(ing.getText())
            
    elif 'MyRecipes' in soup.title.string:
        print('This recipe is from MyRecipes')
        is_recipe=True
        ing_wrap=soup.find_all(itemprop="ingredient")
        for ing in ing_wrap:
            ings.append(ing.getText())

    else:
        is_recipe = False
        ing_wrap=None
        
    if not ing_wrap:  #return warning if website is recognized but format/data extraction is not successful
        print('*******')
        print('NEED NEW FORMAT')  
        print('*******')


    return ing_wrap, is_recipe, ings, ing_amnts


In [9]:
#Nadine's loop for all recipes in folder
# data has following row structure
# RecipeName as Identifier - bool contains_meat - list of co2 ingredients - carbonFootprint - ingredients
data=[]
step=0
for filename in os.listdir(DATA_FOLDER_EXPANDED):
    with open(DATA_FOLDER_EXPANDED+filename) as f:
        isTrue=False
        count_exceptions=0
        print('###############################')
        
        # introduce try/catch such that it does no longer stop, when not recoginzeing letter
        try:
            page = f.read()
            soup = BeautifulSoup(page, 'html.parser')
            print('Title: ', soup.title.string)
            print('filename: ', filename)
            ingredient_wrappers, is_recipe, ingredients, ingredient_amounts = analyse_recipe(soup,page)
            print('is recipe: ',is_recipe)
            #print(ingredients)
            #print(ingredient_amounts)
            #different tag for ingredient and different separators in ingredient list between the webpages
            if is_recipe:
                #Extract all ingredients (and ingredient amounts if available)
                #Extract meat ingredients and quantities in kg
                print('ingredient ammount = ', ingredient_amounts)
                meat_ingredients_full, meat_ingredients_base, ingredient_quant,contains_meat=extract_amount(ingredients, ingredient_amounts, meat_products)
                print('does this recipe contain meat? ', contains_meat)
                print('ingredients = ',ingredients)
                print('meat ingredients=', meat_ingredients_base)
                print('ingredient_quantity (kg)= ',ingredient_quant)
                #data.append
                #add row to dataset only if recipe contains meat
                #data.append([soup.title.string, contains_meat, meatlist, carbon_fp(meatlist), ingredients])
            else:
                print(soup.title.string)
        except:
            print('exception')
            count_exceptions=count_exceptions+1
    step=step+1
    if step>=100:
        break
#print(data)
# save data as csv 

###############################
Title:  Money-Saving Meals : Recipes and Cooking : Food Network
filename:  0a6650d159357825c5ae98eb323dc9f3.html
exception
###############################
exception
###############################
Title:  
	Cheese and Onion Pie Recipe - Allrecipes.com

filename:  00ee3c71a71548d65f4c5a1dd573fbc6.html
exception
###############################
Title:  
	Wonton Wrappers Recipe - Allrecipes.com

filename:  0a33c71ae50009983ff80e13f41f02db.html
exception
###############################
Title:  Cinnamon Bun Scones Recipe #68967 from CDKitchen.com
filename:  0a7349f0a41597144eecac064221457d.html
exception
###############################
Title:  
	Twice Baked Cheesy Potatoes Recipe - Allrecipes.com

filename:  0a86d840f8aa2314221c19e9e0b1a712.html
exception
###############################
Title:  Oreo Cookies And Cream Cupcakes Recipe - Food.com - 258537
filename:  0aaf1336627337d37789775657d32019.html
exception
###############################
Title:  











Title:  
	Liver Pate Recipe - Allrecipes.com

filename:  00d1e7408cd5c21e086f027435bc239f.html
exception
###############################
Title:  
	bush tucker pictures - All recipes Australia NZ

filename:  0a415e8ecc9c766f80c0dfcc53779912.html
exception
###############################
Title:  Cooks.com - Recipes - Easy Pancakes
filename:  0abe9057337ab521097ffcc1e77c1535.html
exception
###############################
Title:  Rolled Turkey Breast With Pancetta, Figs And Leek Recipe - Taste.com.au
filename:  00bb0f80a1286ab29611a34540a8f54e.html
exception
###############################
Title:  Baby Shower Menu Ideas - Food and Recipes for Baby Showers - Delish.com
filename:  0a1356eadc66b16138025e9c0ca8f884.html
exception
###############################
exception
###############################
Title:  Spiced Chicken Tenders with Dipping Sauces Recipe : Sandra Lee : Recipes : Food Network
filename:  0ab52a6d77eed120dbb17f2bbdd82bd3.html
exception
###############################
Title: 

In [ ]:
# calculate carbon footprint
#input ingredients
#identify ingredient ammount
#output carbon footprint
def carbon_fp (l):
    """
    takes a list of ingredients contributing to co2 and returns carbon footprint
    """
    c=len(l)
    return c

